In [ ]:
{
  "Comment": "State machine to process predictions and route to appropriate lambda functions.",
  "StartAt": "EvaluatePrediction",
  "States": {
    "EvaluatePrediction": {
      "Type": "Choice",
      "Choices": [
        {
          "Variable": "$.predicted_source",
          "StringEquals": "cloudshell.amazonaws.com",
          "Next": "InvokeCloudShellFunction"
        },
        {
          "Variable": "$.predicted_source",
          "StringEquals": "ec2.amazonaws.com",
          "Next": "InvokeEC2Function"
        },
        {
          "Variable": "$.predicted_source",
          "StringEquals": "cloudtrail.amazonaws.com",
          "Next": "InvokeCloudTrailFunction"
        }
      ],
      "Default": "HandleUnknownSource"
    },
    "InvokeCloudShellFunction": {
      "Type": "Task",
      "Resource": "arn:aws:lambda:us-east-1::function:CloudShellHandler",
      "Parameters": {
        "detail": {
          "eventName.$": "$.event_name"
        }
      },
      "ResultPath": "$.cloudshell_result",
      "Next": "FinishWorkflow"
    },
    "InvokeEC2Function": {
      "Type": "Task",
      "Resource": "arn:aws:lambda:us-east-1::function:EC2Handler",
      "Parameters": {
        "detail": {
          "eventName.$": "$.event_name"
        }
      },
      "ResultPath": "$.ec2_result",
      "Next": "FinishWorkflow"
    },
    "InvokeCloudTrailFunction": {
      "Type": "Task",
      "Resource": "arn:aws:lambda:us-east-1::function:CloudTrailHandler",
      "Parameters": {
        "detail": {
          "eventName.$": "$.event_name"
        }
      },
      "ResultPath": "$.cloudtrail_result",
      "Next": "FinishWorkflow"
    },
    "HandleUnknownSource": {
      "Type": "Fail",
      "Error": "UnknownSourceError",
      "Cause": "The predicted source is not recognized."
    },
    "FinishWorkflow": {
      "Type": "Succeed"
    }
  }
}